In [119]:
import numpy as np
import trimesh
import tqdm 

In [120]:
def uniform_sampling_from_mesh(vertices, faces, sample_num):
    # -------- TODO -----------
    # 1. compute area of each triangles
    # 2. compute probability of each triangles from areas
    # 3. sample N faces according to the probability
    # 4. for each face, sample 1 point
    # Note that FOR-LOOP is not allowed!
    # -------- TODO -----------
    p0 = vertices[faces[:, 0], :]
    vec1 = vertices[faces[:, 1], :] - vertices[faces[:, 0], :]
    vec2 = vertices[faces[:, 2], :] - vertices[faces[:, 0], :]
    area = 0.5 * np.linalg.norm(np.cross(vec1, vec2), axis=1)
    prob = area / np.sum(area)
    faces_sampled = np.random.choice(faces.shape[0], sample_num, p=prob)
    u, v = np.random.rand(sample_num), np.random.rand(sample_num)
    u, v = np.where(u + v <= 1 , u, 1 - u), np.where(u + v <= 1 , v, 1 - v)
    uniform_pc = p0[faces_sampled, :] + np.reshape(u, (u.shape[0], 1)) * vec1[faces_sampled, :] + np.reshape(v, (v.shape[0], 1)) * vec2[faces_sampled, :]
    return area, prob, uniform_pc
        

In [121]:
def farthest_point_sampling(pc, sample_num):
    # -------- TODO -----------
    # FOR LOOP is allowed here.
    # -------- TODO -----------
    fps = [np.random.randint(pc.shape[0])]
    dis = np.linalg.norm(pc - pc[fps[0], :], axis=1)
    u = (np.arange(pc.shape[0]) == fps[0])
    for _ in range(1, sample_num, 1):
        idx = np.argmax(dis)
        fps.append(idx)
        u[idx] = True
        dis = np.minimum(dis, np.linalg.norm(pc - pc[idx, :], axis=1))
    results = pc[fps, :]
    return results

In [122]:
# task 1: uniform sampling 

obj_path = 'spot.obj'
mesh = trimesh.load(obj_path)
print('faces shape: ', mesh.faces.shape)
sample_num = 512
area, prob, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)

# Visualization. For you to check your code
np.savetxt('uniform_sampling_vis.txt', uniform_pc)

print('area shape: ',area.shape)
print('prob shape: ',prob.shape)
print('pc shape: ',uniform_pc.shape)
# the result should satisfy: 
#       area.shape = (13712, ) 
#       prob.shape = (13712, ) 
#       uniform_pc.shape = (512, 3) 

# For submission
save_dict = {'area': area, 'prob': prob, 'pc': uniform_pc}
np.save('../results/uniform_sampling_results', save_dict)

faces shape:  (13712, 3)
area shape:  (13712,)
prob shape:  (13712,)
pc shape:  (512, 3)


In [123]:
# task 2: FPS

init_sample_num = 2000
final_sample_num = 512
_,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)

# Visualization. For you to check your code
np.savetxt('fps_vis.txt', fps_pc)

# For submission
np.save('../results/fps_results', fps_pc)

In [124]:
# task 3: metrics

import scipy
from scipy.optimize import linear_sum_assignment
# -----------TODO---------------
# compute chamfer distance and EMD for two point clouds sampled by uniform sampling and FPS.
# sample and compute CD and EMD again. repeat for five times.
# save the mean and var.
# -----------TODO---------------

CD_mean = 0
CD_var = 0
EMD_mean = 0
EMD_var = 0
rep_num = 5
CD = np.zeros((rep_num,))
EMD = np.zeros((rep_num, ))
for rp in range(rep_num):
    _,_, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)
    _,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
    fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)
    
    dis = np.linalg.norm(np.reshape(uniform_pc, (uniform_pc.shape[0], 1, 3)) - fps_pc, axis=2)
    CD[rp] = (np.average(np.amin(dis, axis=1)) + np.average(np.amin(dis, axis=0))) * 0.5
    row_ind, col_ind = linear_sum_assignment(dis)
    EMD[rp] = np.average(dis[row_ind, col_ind])

CD_mean = np.mean(CD)
CD_var = np.var(CD)
EMD_mean = np.mean(EMD)
EMD_var = np.var(EMD)
print(CD_mean, CD_var, EMD_mean, EMD_var)

# For submission
np.save('../results/metrics', {'CD_mean':CD_mean, 'CD_var':CD_var, 'EMD_mean':EMD_mean, 'EMD_var':EMD_var})

1.3884228529136031 0.0002291958584138996 2.4587069497924663 0.008586021589384776
